In [1]:
import sent2vec
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from scipy.spatial import distance
import pickle
from tqdm import tqdm
import numpy as np

In [2]:
model_path = '/Users/jplasser/Downloads/BioSentVec_PubMed_MIMICIII-bigram_d700.bin'
model = sent2vec.Sent2vecModel()
try:
    model.load_model(model_path)
except Exception as e:
    print(e)
print('model successfully loaded')

model successfully loaded


In [3]:
stop_words = set(stopwords.words('english'))
def preprocess_sentence(text):
    text = text.replace('/', ' / ')
    text = text.replace('.-', ' .- ')
    text = text.replace('.', ' . ')
    text = text.replace('\'', ' \' ')
    text = text.lower()

    tokens = [token for token in word_tokenize(text) if token not in punctuation and token not in stop_words]

    return ' '.join(tokens)

In [13]:
seq_len = None # 2000

# data_path = '/Users/jplasser/Documents/AI Master/WS2021/MastersThesis/code.nosync/CNEP/src/data/mimic3/'
data_path = './'

datasets = ['train','val','test']

for dataset in datasets:
    embeds = []
    train_data = pickle.load(open(f'{data_path}new_{dataset}_data_unique.pickle', 'rb'))

    for i in tqdm(range(len(train_data['notes']))):
        sentence_vector = model.embed_sentence(preprocess_sentence(train_data['notes'][i][:seq_len]))
        embeds.append(sentence_vector.reshape(-1))
        
    embeds = np.array(embeds)
    print(train_data['inputs'].shape, embeds.shape)
    train_data['embeds'] = embeds
    pickle.dump(train_data, open(f'{data_path}new_{dataset}_data_unique_embed.pickle', 'wb'))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13781/13781 [01:12<00:00, 190.90it/s]


(13781, 48, 390) (13781, 700)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4403/4403 [00:24<00:00, 180.22it/s]


(4403, 48, 390) (4403, 700)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4424/4424 [00:32<00:00, 137.70it/s]


(4424, 48, 390) (4424, 700)


# if there are the CNEP embeddings in the data sets

In [5]:
seq_len = None # 2000

# data_path = '/Users/jplasser/Documents/AI Master/WS2021/MastersThesis/code.nosync/CNEP/src/data/mimic3/'
data_path = './'

datasets = ['train','val','test']

for dataset in datasets:
    embeds = []
    embeds_events = []
    train_data = pickle.load(open(f'{data_path}new_{dataset}_data_unique_CNEP.pickle', 'rb'))

    for i in tqdm(range(len(train_data['notes']))):
        sentence_vector = model.embed_sentence(preprocess_sentence(train_data['notes'][i][:seq_len]))
        embeds.append(sentence_vector.reshape(-1))
    
    for i in tqdm(range(len(train_data['eventsnotes']))):
        sentence_vector = model.embed_sentence(preprocess_sentence(train_data['eventsnotes'][i][:seq_len]))
        embeds_events.append(sentence_vector.reshape(-1))
        
    embeds = np.array(embeds)
    embeds_events = np.array(embeds_events)
    print(train_data['inputs'].shape, embeds.shape, embeds_events.shape)
    train_data['embeds'] = embeds
    train_data['embeds_events'] = embeds_events
    del train_data['notes']
    del train_data['eventsnotes']
    pickle.dump(train_data, open(f'{data_path}new_{dataset}_data_unique_embed_CNEP.pickle', 'wb'))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13181/13181 [03:02<00:00, 72.34it/s]


(13181, 48, 390) (13181, 700) (13181, 700)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4216/4216 [00:45<00:00, 93.26it/s]


(4216, 48, 390) (4216, 700) (4216, 700)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4204/4204 [00:45<00:00, 91.48it/s]


(4204, 48, 390) (4204, 700) (4204, 700)
